# `mammos_entity` quickstart

## Some definitions

- a `mammos_units.Quantity` is an object that carries a value and a unit.

- a `mammos_entity.Entity` links a quantity to its definition in the [EMMO ontology](https://github.com/emmo-repo/EMMO) and the [MaMMoS additions for magnetic materials](https://github.com/MaMMoS-project/MagneticMaterialsOntology)


In [1]:
import mammos_entity as me

## Creating entities

For entities that are important in MaMMoS, there are [convenient attributes](https://mammos-project.github.io/mammos/api/mammos_entity.html) to define those. For example for the saturation magnetization Ms:

In [2]:
Ms = me.Ms(800e3)  # defines Ms = 8e5 A/m  (default units are SI, i.e. A/m here)

In [3]:
Ms

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(800000.0), unit='A / m')

If no units are provided, then the ontology units are used, such as `A/m` in the example above. These units are SI units typically without numerical prefactors (such as kilo, milli, etc.)

If units are provided, these are compared with the units expected for that entity. An error is raised if they do not match. This is good practice for extra clarity.

In [4]:
M1 = me.Ms(800e3, "A/m")
M1

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(800000.0), unit='A / m')

Providing units can be also useful, if numerical prefactors are used (such as kilo):

In [5]:
M2 = me.Ms(800, "kA/m")
M2

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(800.0), unit='kA / m')

In [6]:
M1 == M2

True

We can create an entity from a Quantity as well:

In [7]:
import mammos_units as u

Ms_quantity = 800 * u.kA / u.m
Ms_quantity

<Quantity 800. kA / m>

In [8]:
me.Ms(Ms_quantity)

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(800.0), unit='kA / m')

If we like to express spontaneous magnetization in Tesla, we can convert it to the required A/m. To allow that conversion we must explicitely enable the `magnetic_flux_field` equivalency ($B=\mu_0H$):

In [9]:
# enable implicit conversion between A/m and T in the rest of the notebook:
u.set_enabled_equivalencies(u.magnetic_flux_field())

a = 1.2 * u.T  # Quantity in Tesla

Ms = me.Ms(a.to("A/m"))  # Convert Quantity to A/m and create entity
Ms

Entity(ontology_label='SpontaneousMagnetization', value=np.float64(954929.6580315315), unit='A / m')

As a cross-check we can access the quantity attribute and convert that back to Tesla. This operation does not modify `Ms`.

In [10]:
Ms.q.to("T")

<Quantity 1.2 T>

Initializing with incompatible units fails with an error. In particular, implicit conversion via equivalencies is not supported. Convert to the required units first, as shown above.

In [11]:
me.Ms(1.2, "T")  # Tesla not compatible with A/m

UnitConversionError: The unit 'T' is not equivalent to the unit of SpontaneousMagnetization 'A / m'

For more details about unit conversion and equivalencies refer to the documentation of [mammos-units](https://mammos-project.github.io/mammos/examples/mammos-units/example.html#Equivalency).

## Access to ontology

Each entity object knows its label in the ontology:

In [12]:
Ms.ontology_label

'SpontaneousMagnetization'

The ontology attribute links to an [owlready2](https://github.com/pwin/owlready2) object created by the [EMMOntoPy](https://emmo-repo.github.io/EMMOntoPy/stable/) package.

When saving data to a file, the attribute `ontology_label_with_iri` might be useful to save in the metadata as it returns a string containing the ontology label together with the unique identifier of the ontology entry (IRI is the Internationalized Resource Identifier).

In [13]:
Ms.ontology_label_with_iri

'SpontaneousMagnetization https://w3id.org/emmo/domain/magnetic_material#EMMO_032731f8-874d-5efb-9c9d-6dafaa17ef25'

We can use all attributes of the ontology object through `Ms.ontology`:

In [14]:
Ms.ontology.get_annotations()

{'prefLabel': [locstr('SpontaneousMagnetization', 'en')],
 'altLabel': ['Ms'],
 'elucidation': [locstr('The spontaneous magnetization, Ms, of a ferromagnet is the result\nof alignment of the magnetic moments of individual atoms. Ms exists\nwithin a domain of a ferromagnet.', 'en')],
 'IECEntry': ['https://www.electropedia.org/iev/iev.nsf/display?openform&ievref=221-02-41'],
 'wikipediaReference': ['https://en.wikipedia.org/wiki/Spontaneous_magnetization']}

In [15]:
Ms.ontology.get_class_properties()

{magnetic_material.wikipediaReference,
 emmo.elucidation,
 core.altLabel,
 core.prefLabel,
 magnetic_material.IECEntry,
 emmo.hasMeasurementUnit}

In [16]:
print(Ms.ontology.elucidation[0])

The spontaneous magnetization, Ms, of a ferromagnet is the result
of alignment of the magnetic moments of individual atoms. Ms exists
within a domain of a ferromagnet.


## Entities contain Quantities

Each entity has a `quantity` attribute containing the `mammos_units.Quantity` object. See [mammos_units examples](https://mammos-project.github.io/mammos/examples/mammos-units/index.html) for details.

In [17]:
Ms.quantity

<Quantity 954929.65803153 A / m>

There is also the shorthand `q` for the `quantity` attribute:

In [18]:
Ms.q

<Quantity 954929.65803153 A / m>

The attributes `value` and `unit` provide direct access to value and unit of the quantity:

In [19]:
Ms.value

np.float64(954929.6580315315)

In [20]:
Ms.unit

Unit("A / m")

Entities do not support numerical operations. To perform these operations, we first need to extract the quantity:

In [21]:
Ms.q**2

<Quantity 9.11890652e+11 A2 / m2>

In [22]:
Ms.quantity.to("kA/m")

<Quantity 954.92965803 kA / m>

In [23]:
Ms.quantity.to("T")

<Quantity 1.2 T>

## Defining vector entities (Example Zeeman field)

In [24]:
H = me.H([1e4, 1e4, 1e4], "A/m")
H

Entity(ontology_label='ExternalMagneticField', value=array([10000., 10000., 10000.]), unit='A / m')

In [25]:
H.ontology

magnetic_material.ExternalMagneticField

In [26]:
print(H.ontology.elucidation[0])

The external field H′, acting on a sample that is produced by
electric currents or the stray field of magnets outside the sample
volume, is often called the applied field.


## Does `mammos_entity` not provide your preferred entity?

The list of convenience attributes is at 
https://mammos-project.github.io/mammos/api/mammos_entity.html

If the desired entity is not available, we can search the EMMO ontology (including the `magnetic_material_mammos` additions), for example for entity labels containing the string "Magnetization":

In [27]:
me.mammos_ontology.get_by_label_all("*Field*")

{emmo.ElectricDisplacementFieldUnit,
 emmo.ElectricFieldStrength,
 emmo.ElectricFieldStrengthUnit,
 emmo.MagneticFieldStrength,
 emmo.MagneticFieldStrengthUnit,
 magnetic_material.AnisotropyField,
 magnetic_material.DemagnetizingField,
 magnetic_material.ExternalMagneticField,
 magnetic_material.InternalMagneticField,
 magnetic_material.KneeField,
 magnetic_material.KneeFieldExternal,
 magnetic_material.SwitchingFieldCoercivity,
 magnetic_material.SwitchingFieldCoercivityExternal}

Once identified the right label, we create an entity like this:

In [28]:
me.Entity("ElectricFieldStrength", value=230)

Entity(ontology_label='ElectricFieldStrength', value=np.float64(230.0), unit='V / m')